In [22]:
# Cell 1: Setup & Configuration (UPDATED)
import numpy as np
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass, label
import pandas as pd
from scipy.stats import pearsonr
from sklearn.manifold import MDS
from scipy.spatial import procrustes
import seaborn as sns
from scipy.stats import linregress

# Use the CSV-driven configuration from the functional extraction notebook
BASE_DIR = Path("/user_data/csimmon2/long_pt")
OUTPUT_DIR = BASE_DIR / "analyses" / "rsa_corrected"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load subject info from CSV (same as functional extraction notebook)
CSV_FILE = Path('/user_data/csimmon2/git_repos/long_pt/long_pt_sub_info.csv')
df = pd.read_csv(CSV_FILE)
SESSION_START = {'sub-010': 2, 'sub-018': 2, 'sub-068': 2}

# Use the same subject loading function
def load_subjects_by_group(group_filter=None, patient_only=True):
    """Load subjects dynamically from CSV"""
    filtered_df = df.copy()
    
    if patient_only is True:
        filtered_df = filtered_df[filtered_df['patient'] == 1]
    elif patient_only is False:
        filtered_df = filtered_df[filtered_df['patient'] == 0]
    
    if group_filter:
        if isinstance(group_filter, str):
            group_filter = [group_filter]
        filtered_df = filtered_df[filtered_df['group'].isin(group_filter)]
    
    subjects = {}
    
    for _, row in filtered_df.iterrows():
        subject_id = row['sub']
        
        subj_dir = BASE_DIR / subject_id
        if not subj_dir.exists():
            continue
            
        sessions = []
        for ses_dir in subj_dir.glob('ses-*'):
            if ses_dir.is_dir():
                sessions.append(ses_dir.name.replace('ses-', ''))
        
        if not sessions:
            continue
            
        sessions = sorted(sessions, key=lambda x: int(x))
        start_session = SESSION_START.get(subject_id, 1)
        available_sessions = [s for s in sessions if int(s) >= start_session]
        
        if not available_sessions:
            continue
            
        hemisphere_full = row.get('intact_hemi', 'left') if pd.notna(row.get('intact_hemi', None)) else 'left'
        hemisphere = 'l' if hemisphere_full == 'left' else 'r'
        
        subjects[subject_id] = {
            'code': f"{row['group']}{subject_id.split('-')[1]}",
            'sessions': available_sessions,
            'hemi': hemisphere,
            'group': row['group'],
            'patient_status': 'patient' if row['patient'] == 1 else 'control',
            'age_1': row['age_1'] if pd.notna(row['age_1']) else None
        }
    
    return subjects

# Reload with correct hemisphere mapping
ALL_PATIENTS = load_subjects_by_group(group_filter=None, patient_only=True)
OTC_PATIENTS = load_subjects_by_group(group_filter='OTC', patient_only=True)
NON_OTC_PATIENTS = load_subjects_by_group(group_filter='nonOTC', patient_only=True)
ALL_CONTROLS = load_subjects_by_group(group_filter=None, patient_only=False)
ALL_SUBJECTS = {**ALL_PATIENTS, **ALL_CONTROLS} # Combine patients and controls

# Update analysis subjects
ANALYSIS_SUBJECTS = ALL_SUBJECTS

# NOTE ROI COPE MAP HERE MATCHES FUNCTIONAL EXTRACTION
#COPE_MAP = {
#    'face': 10,  # Updated to match functional extraction
#    'word': 12,
#    'object': 3,
#    'house': 11  # Updated to match functional extraction
#}


# COPE MAP HERE IS THE ONE WE WANT TO USE FOR RSA
COPE_MAP = {
    'face': 1,
    'word': 12,
    'object': 3,
    'house': 2
}

print(f"RSA Analysis - {len(ANALYSIS_SUBJECTS)} subjects loaded")
print("Subjects:", list(ANALYSIS_SUBJECTS.keys()))

RSA Analysis - 25 subjects loaded
Subjects: ['sub-004', 'sub-007', 'sub-008', 'sub-010', 'sub-017', 'sub-021', 'sub-045', 'sub-047', 'sub-049', 'sub-070', 'sub-072', 'sub-073', 'sub-079', 'sub-081', 'sub-086', 'sub-108', 'sub-018', 'sub-022', 'sub-025', 'sub-027', 'sub-052', 'sub-058', 'sub-062', 'sub-064', 'sub-068']


In [27]:
# Cell 2: Extract Functional ROIs (INTEGRATED)
from scipy.ndimage import label, center_of_mass

def extract_functional_rois_final(subject_id, subjects_dict, threshold_z=2.3):
    """Extract functional cluster ROIs across all sessions"""
    
    if subject_id not in subjects_dict:
        print(f"❌ {subject_id} not in analysis group")
        return {}
        
    info = subjects_dict[subject_id]
    code = info['code']
    hemi = info['hemi']
    sessions = info['sessions']
    first_session = sessions[0]
    
    print(f"\n{code} - Extracting Functional ROIs [{info['group']} {info['patient_status']}]")
    
    all_results = {}
    
    # Process each category using COPE_MAP
    for category, cope_num in COPE_MAP.items():
        all_results[category] = {}
        
        # Load category-specific mask
        mask_file = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs' / f'{hemi}_{category}_searchmask.nii.gz'
        if not mask_file.exists():
            print(f"  ⚠️  {category}: mask not found")
            continue
        
        mask = nib.load(mask_file).get_fdata() > 0
        affine = nib.load(mask_file).affine
        
        # Process each session
        for session in sessions:
            feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
            
            # Select correct zstat file
            zstat_file = 'zstat1.nii.gz' if session == first_session else f'zstat1_ses{first_session}.nii.gz'
            cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / zstat_file
            
            if not cope_file.exists():
                continue
            
            # Load functional activation
            zstat = nib.load(cope_file).get_fdata()
            suprathresh = (zstat > threshold_z) & mask
            
            if suprathresh.sum() < 50:
                continue
            
            # Find largest cluster
            labeled, n_clusters = label(suprathresh)
            if n_clusters == 0:
                continue
                
            cluster_sizes = [(labeled == i).sum() for i in range(1, n_clusters + 1)]
            largest_idx = np.argmax(cluster_sizes) + 1
            roi_mask = (labeled == largest_idx)
            
            # Extract metrics
            peak_idx = np.unravel_index(np.argmax(zstat * roi_mask), zstat.shape)
            peak_z = zstat[peak_idx]
            centroid = nib.affines.apply_affine(affine, center_of_mass(roi_mask))
            
            # Store results
            all_results[category][session] = {
                'n_voxels': cluster_sizes[largest_idx - 1],
                'peak_z': peak_z,
                'centroid': centroid,
                'roi_mask': roi_mask
            }
    
    return all_results

# Extract functional ROIs for all analysis subjects
print("="*70)
print("EXTRACTING FUNCTIONAL ROIs FOR RSA ANALYSIS")
print("="*70)

golarai_functional_final = {}

for subject_id in ANALYSIS_SUBJECTS.keys():
    try:
        golarai_functional_final[subject_id] = extract_functional_rois_final(subject_id, ANALYSIS_SUBJECTS, threshold_z=2.3)
    except Exception as e:
        print(f"❌ {subject_id} failed: {e}")
        golarai_functional_final[subject_id] = {}

print(f"\n✓ Functional ROI extraction complete for {len(golarai_functional_final)} subjects!")

# Quick summary
for subject_id, results in golarai_functional_final.items():
    if subject_id in ANALYSIS_SUBJECTS:
        code = ANALYSIS_SUBJECTS[subject_id]['code']
        categories = list(results.keys())
        total_sessions = sum(len(sessions) for sessions in results.values())
        print(f"  {code}: {len(categories)} categories, {total_sessions} total ROIs")

EXTRACTING FUNCTIONAL ROIs FOR RSA ANALYSIS

OTC004 - Extracting Functional ROIs [OTC patient]

nonOTC007 - Extracting Functional ROIs [nonOTC patient]

OTC008 - Extracting Functional ROIs [OTC patient]

OTC010 - Extracting Functional ROIs [OTC patient]

OTC017 - Extracting Functional ROIs [OTC patient]

OTC021 - Extracting Functional ROIs [OTC patient]

nonOTC045 - Extracting Functional ROIs [nonOTC patient]

nonOTC047 - Extracting Functional ROIs [nonOTC patient]

nonOTC049 - Extracting Functional ROIs [nonOTC patient]

nonOTC070 - Extracting Functional ROIs [nonOTC patient]

nonOTC072 - Extracting Functional ROIs [nonOTC patient]

nonOTC073 - Extracting Functional ROIs [nonOTC patient]

OTC079 - Extracting Functional ROIs [OTC patient]

nonOTC081 - Extracting Functional ROIs [nonOTC patient]

nonOTC086 - Extracting Functional ROIs [nonOTC patient]

OTC108 - Extracting Functional ROIs [OTC patient]
  ⚠️  face: mask not found
  ⚠️  word: mask not found
  ⚠️  object: mask not found
  ⚠

In [28]:
# Cell 3: RSA Helper Functions (UPDATED)
def create_6mm_sphere(peak_coord, affine, brain_shape, radius=6):
    """Create a 6mm sphere around a peak."""
    grid_coords = np.array(np.meshgrid(
        np.arange(brain_shape[0]), 
        np.arange(brain_shape[1]), 
        np.arange(brain_shape[2]),
        indexing='ij'
    )).reshape(3, -1).T
    
    grid_world = nib.affines.apply_affine(affine, grid_coords)
    distances = np.linalg.norm(grid_world - peak_coord, axis=1)
    
    mask_3d = np.zeros(brain_shape, dtype=bool)
    within = grid_coords[distances <= radius]
    for coord in within:
        mask_3d[coord[0], coord[1], coord[2]] = True
    
    return mask_3d

def extract_beta_patterns_from_sphere(subject_id, session, sphere_mask, category_copes):
    """Extract beta values from a 6mm sphere for all categories."""
    info = ANALYSIS_SUBJECTS[subject_id]  # UPDATED: Use ANALYSIS_SUBJECTS
    first_session = info['sessions'][0]
    
    feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
    
    beta_patterns = []
    valid_categories = []
    
    for category, cope_num in category_copes.items():
        # UPDATED: Use correct file naming based on session
        if session == first_session:
            cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / 'cope1.nii.gz'
        else:
            cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / f'cope1_ses{first_session}.nii.gz'
        
        if not cope_file.exists():
            continue
        
        cope_data = nib.load(cope_file).get_fdata()
        roi_betas = cope_data[sphere_mask]
        roi_betas = roi_betas[np.isfinite(roi_betas)]
        
        if len(roi_betas) > 0:
            beta_patterns.append(roi_betas)
            valid_categories.append(category)
    
    if len(beta_patterns) == 0:
        return None, None
    
    min_voxels = min(len(b) for b in beta_patterns)
    beta_patterns = [b[:min_voxels] for b in beta_patterns]
    beta_matrix = np.column_stack(beta_patterns)
    
    return beta_matrix, valid_categories

def compute_rdm(beta_matrix, fisher_transform=True):
    """Compute Representational Dissimilarity Matrix."""
    correlation_matrix = np.corrcoef(beta_matrix.T)
    rdm = 1 - correlation_matrix
    
    if fisher_transform:
        correlation_matrix_fisher = np.arctanh(np.clip(correlation_matrix, -0.999, 0.999))
        return rdm, correlation_matrix_fisher
    else:
        return rdm, correlation_matrix

print("✓ RSA helper functions updated")

✓ RSA helper functions updated


In [ ]:
# Cell 4: Session-Specific RSA Analysis (UPDATED)
def extract_all_rdms_6mm_session_specific(functional_results, analysis_subjects):
    """Extract RDMs from 6mm spheres using SESSION-SPECIFIC ROIs - UPDATED"""
    all_rdms = {}
    
    for subject_id in analysis_subjects.keys():  # UPDATED: Use analysis_subjects
        if subject_id not in functional_results:
            continue
            
        info = analysis_subjects[subject_id]  # UPDATED: Use analysis_subjects
        code = info['code']
        hemi = info['hemi']
        sessions = info['sessions']
        first_session = sessions[0]
        
        # UPDATED: Use first session for reference
        ref_file = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs' / f'{hemi}_face_searchmask.nii.gz'
        if not ref_file.exists():
            print(f"⚠️ Reference file missing for {code}: {ref_file}")
            continue
            
        ref_img = nib.load(ref_file)
        affine = ref_img.affine
        brain_shape = ref_img.shape
        
        print(f"\n{code} ({info['group']} {info['patient_status']}): SESSION-SPECIFIC RSA Analysis")
        
        all_rdms[subject_id] = {}
        
        for roi_name in ['face', 'word', 'object', 'house']:
            if roi_name not in functional_results[subject_id]:
                continue
            
            all_rdms[subject_id][roi_name] = {
                'rdms': {},
                'correlation_matrices': {},
                'beta_patterns': {},
                'valid_categories': None,
                'session_peaks': {},
                'session_n_voxels': {}
            }
            
            for session in sessions:
                if session not in functional_results[subject_id][roi_name]:
                    continue
                
                # SESSION-SPECIFIC peak and sphere
                peak = functional_results[subject_id][roi_name][session]['centroid']
                sphere_mask = create_6mm_sphere(peak, affine, brain_shape, radius=6)
                n_voxels = sphere_mask.sum()
                
                all_rdms[subject_id][roi_name]['session_peaks'][session] = peak
                all_rdms[subject_id][roi_name]['session_n_voxels'][session] = n_voxels
                
                beta_matrix, valid_cats = extract_beta_patterns_from_sphere(
                    subject_id, session, sphere_mask, COPE_MAP  # UPDATED: Use COPE_MAP
                )
                
                if beta_matrix is None:
                    continue
                
                rdm, corr_matrix_fisher = compute_rdm(beta_matrix, fisher_transform=True)
                
                all_rdms[subject_id][roi_name]['rdms'][session] = rdm
                all_rdms[subject_id][roi_name]['correlation_matrices'][session] = corr_matrix_fisher
                all_rdms[subject_id][roi_name]['beta_patterns'][session] = beta_matrix
                all_rdms[subject_id][roi_name]['valid_categories'] = valid_cats
                
                print(f"  {roi_name} ses-{session}: {n_voxels} voxels")
    
    return all_rdms

# Run with session-specific ROIs
print("STEP 1: SESSION-SPECIFIC ROI RSA")
print("="*70)
rsa_rdms_6mm_session_specific = extract_all_rdms_6mm_session_specific(golarai_functional_final, ANALYSIS_SUBJECTS)
print("✓ Session-specific analysis complete!")

STEP 1: SESSION-SPECIFIC ROI RSA

OTC004 (OTC patient): SESSION-SPECIFIC RSA Analysis
  face ses-02: 906 voxels
  face ses-03: 903 voxels
  face ses-05: 899 voxels
  face ses-06: 903 voxels
  word ses-02: 898 voxels
  word ses-03: 904 voxels
  word ses-05: 913 voxels
  word ses-06: 898 voxels
  object ses-01: 905 voxels
  object ses-02: 906 voxels
  object ses-03: 903 voxels
  object ses-05: 911 voxels
  object ses-06: 904 voxels
  house ses-01: 897 voxels
  house ses-02: 918 voxels
  house ses-03: 914 voxels
  house ses-05: 900 voxels
  house ses-06: 902 voxels

nonOTC007 (nonOTC patient): SESSION-SPECIFIC RSA Analysis


In [6]:
# CELL 4: CORRECTED SESSION-SPECIFIC RSA ANALYSIS (SPACE-MATCHED)

def extract_all_rdms_6mm_session_specific_cross_validated(functional_results):
    """Extract RDMs using SESSION-SPECIFIC ROIs and space-matched cross-validated RSA."""
    all_rdms = {}
    category_copes = {'face': 1, 'word': 12, 'object': 3, 'house': 2}
    
    for subject_id in ['sub-004', 'sub-021']:
        code = SUBJECTS[subject_id]['code']
        sessions = SUBJECTS[subject_id]['sessions']
        
        print(f"\n{code}: SPACE-MATCHED CROSS-VALIDATED RSA Analysis")
        
        all_rdms[subject_id] = {}
        
        for roi_name in ['face', 'word', 'object', 'house']:
            if roi_name not in functional_results[subject_id]:
                continue
            
            all_rdms[subject_id][roi_name] = {
                'rdms': {},
                'correlation_matrices': {},
                'beta_patterns': {},
                'valid_categories': None,
                'session_peaks': {},
                'session_n_voxels': {}
            }
            
            for session in sessions:
                if session not in functional_results[subject_id][roi_name]:
                    continue
                
                # Get SESSION-SPECIFIC peak coordinate
                peak = functional_results[subject_id][roi_name][session]['centroid']
                
                all_rdms[subject_id][roi_name]['session_peaks'][session] = peak
                
                # SPACE-MATCHED cross-validated beta extraction
                beta_matrix, valid_cats = extract_beta_patterns_from_sphere_cross_validated(
                    subject_id, session, peak, category_copes
                )
                
                if beta_matrix is None:
                    print(f"  {roi_name} ses-{session}: No cross-validated data")
                    continue
                
                rdm, corr_matrix_fisher = compute_rdm(beta_matrix, fisher_transform=True)
                
                all_rdms[subject_id][roi_name]['rdms'][session] = rdm
                all_rdms[subject_id][roi_name]['correlation_matrices'][session] = corr_matrix_fisher
                all_rdms[subject_id][roi_name]['beta_patterns'][session] = beta_matrix
                all_rdms[subject_id][roi_name]['valid_categories'] = valid_cats
                all_rdms[subject_id][roi_name]['session_n_voxels'][session] = beta_matrix.shape[0]
                
                print(f"  {roi_name} ses-{session}: {beta_matrix.shape[0]} voxels, {beta_matrix.shape[1]} categories (cross-validated)")
    
    return all_rdms

# Run space-matched cross-validated analysis
print("STEP 1: SPACE-MATCHED CROSS-VALIDATED RSA")
print("="*70)
rsa_rdms_6mm_cross_validated = extract_all_rdms_6mm_session_specific_cross_validated(golarai_functional_final)
print("✓ Cross-validated analysis complete!")

STEP 1: SPACE-MATCHED CROSS-VALIDATED RSA

UD: SPACE-MATCHED CROSS-VALIDATED RSA Analysis
  face ses-01: 65 voxels, 4 categories (cross-validated)
  face ses-02: 59 voxels, 4 categories (cross-validated)
  face ses-03: 63 voxels, 4 categories (cross-validated)
  face ses-05: 58 voxels, 4 categories (cross-validated)
  face ses-06: 113 voxels, 4 categories (cross-validated)
  word ses-02: 59 voxels, 4 categories (cross-validated)
  word ses-03: 58 voxels, 4 categories (cross-validated)
  word ses-05: 56 voxels, 4 categories (cross-validated)
  word ses-06: 115 voxels, 4 categories (cross-validated)
  object ses-01: 58 voxels, 4 categories (cross-validated)
  object ses-02: 56 voxels, 4 categories (cross-validated)
  object ses-03: 58 voxels, 4 categories (cross-validated)
  object ses-05: 4 voxels, 4 categories (cross-validated)
  object ses-06: 116 voxels, 4 categories (cross-validated)
  house ses-01: 58 voxels, 4 categories (cross-validated)
  house ses-02: 57 voxels, 4 categories (c

In [ ]:
# Cell 5: Liu's RSA Methodology (UPDATED)
def compute_liu_distinctiveness(all_rdms, analysis_subjects):
    """Compute Liu's preferred vs non-preferred category correlations - UPDATED"""
    roi_preferred = {'face': 'face', 'word': 'word', 'object': 'object', 'house': 'house'}
    
    distinctiveness_results = {}
    
    for subject_id, categories in all_rdms.items():
        if subject_id not in analysis_subjects:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        group_status = f"{info['group']} {info['patient_status']}"
        
        distinctiveness_results[subject_id] = {}
        
        print(f"\n{code} ({group_status}): Liu's Distinctiveness Analysis")
        
        for roi_name, roi_data in categories.items():
            if not roi_data['correlation_matrices']:
                continue
            
            valid_cats = roi_data['valid_categories']
            if valid_cats is None or len(valid_cats) < 4:
                continue
            
            preferred_cat = roi_preferred[roi_name]
            if preferred_cat not in valid_cats:
                continue
            
            pref_idx = valid_cats.index(preferred_cat)
            nonpref_indices = [i for i, cat in enumerate(valid_cats) if cat != preferred_cat]
            
            distinctiveness_results[subject_id][roi_name] = {}
            
            for session, corr_matrix in roi_data['correlation_matrices'].items():
                pref_vs_nonpref = corr_matrix[pref_idx, nonpref_indices]
                mean_corr = np.mean(pref_vs_nonpref)
                
                distinctiveness_results[subject_id][roi_name][session] = {
                    'liu_distinctiveness': mean_corr,
                    'individual_correlations': pref_vs_nonpref
                }
                
                print(f"  {roi_name} ses-{session}: {mean_corr:.3f}")
    
    return distinctiveness_results

def compute_liu_bootstrapped_slopes(all_rdms, analysis_subjects, n_bootstraps=1000):
    """Liu's bootstrapped linear regression for dissimilarity changes - UPDATED"""
    from scipy.stats import linregress
    
    slope_results = {}
    pairs = [('face', 'word'), ('house', 'object'), ('face', 'object'), ('word', 'house')]
    
    for subject_id, categories in all_rdms.items():
        if subject_id not in analysis_subjects:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        sessions = info['sessions']
        
        slope_results[subject_id] = {}
        
        print(f"\n{code}: Liu's Bootstrapped Slope Analysis")
        
        for roi_name, roi_data in categories.items():
            if not roi_data['rdms']:
                continue
            
            valid_cats = roi_data['valid_categories']
            if valid_cats is None or len(valid_cats) < 4:
                continue
            
            slope_results[subject_id][roi_name] = {}
            
            for cat1, cat2 in pairs:
                if cat1 not in valid_cats or cat2 not in valid_cats:
                    continue
                
                idx1, idx2 = valid_cats.index(cat1), valid_cats.index(cat2)
                
                session_nums, dissims = [], []
                for session in sessions:
                    if session in roi_data['rdms']:
                        rdm = roi_data['rdms'][session]
                        dissim = rdm[idx1, idx2]
                        session_nums.append(int(session))
                        dissims.append(dissim)
                
                if len(session_nums) < 2:
                    continue
                
                slope = linregress(session_nums, dissims)[0]
                
                null_slopes = []
                for _ in range(n_bootstraps):
                    shuffled = np.random.permutation(dissims)
                    null_slope = linregress(session_nums, shuffled)[0]
                    null_slopes.append(null_slope)
                
                ci_lower = np.percentile(null_slopes, 2.5)
                ci_upper = np.percentile(null_slopes, 97.5)
                significant = slope < ci_lower or slope > ci_upper
                
                slope_results[subject_id][roi_name][f'{cat1}-{cat2}'] = {
                    'observed_slope': slope,
                    'null_slopes': null_slopes,
                    'ci_95': (ci_lower, ci_upper),
                    'significant': significant
                }
                
                sig = "***" if significant else "n.s."
                print(f"  {roi_name} {cat1}-{cat2}: {slope:.4f} {sig}")
    
    return slope_results

# Run Liu's analyses (UPDATED to use ANALYSIS_SUBJECTS)
print("STEP 2: LIU'S RSA METHODOLOGY")
print("="*70)
liu_distinctiveness = compute_liu_distinctiveness(rsa_rdms_6mm_session_specific, ANALYSIS_SUBJECTS)
liu_slopes = compute_liu_bootstrapped_slopes(rsa_rdms_6mm_session_specific, ANALYSIS_SUBJECTS)
print("✓ Liu's methodology complete!")

STEP 2: LIU'S RSA METHODOLOGY

OTC004 (OTC patient): Liu's Distinctiveness Analysis
  face ses-02: 0.346
  face ses-03: -0.025
  face ses-05: 0.103
  face ses-06: 0.128
  word ses-02: 0.097
  word ses-03: 0.658
  word ses-05: 0.246
  word ses-06: 0.179
  object ses-01: -0.352
  object ses-02: -0.891
  object ses-03: -0.293
  object ses-05: -0.531
  object ses-06: -0.451
  house ses-01: -0.928
  house ses-02: -0.872
  house ses-03: -0.369
  house ses-05: -1.008
  house ses-06: -0.441

nonOTC007 (nonOTC patient): Liu's Distinctiveness Analysis
  face ses-01: 0.275
  face ses-03: 0.113
  face ses-04: 0.235
  word ses-03: -0.059
  word ses-04: -0.043
  object ses-01: -0.233
  object ses-03: 0.229
  object ses-04: -0.125
  house ses-01: -0.256
  house ses-03: -0.700
  house ses-04: -0.163

OTC008 (OTC patient): Liu's Distinctiveness Analysis
  face ses-01: 0.372
  face ses-02: 0.469
  word ses-02: -0.023
  object ses-01: 0.801
  object ses-02: 0.052
  house ses-01: -0.081
  house ses-02: -0

In [9]:
# CELL: CALCULATE MEASUREMENT ERROR RADII (UPDATED)
def get_bootstrapped_error_radius(pair_peaks, n_bootstraps=1000):
    """Calculate bootstrapped measurement error radius."""
    if not pair_peaks or len(pair_peaks) < 2:
        return 1.0
    
    data = np.array([p['coord'][:2] for p in pair_peaks])
    
    def stat_func(coords):
        if len(np.unique(coords[:, 0])) < 2 or len(np.unique(coords[:, 1])) < 2:
            return 0.0
        return np.sqrt(np.std(coords[:, 0])**2 + np.std(coords[:, 1])**2)
    
    bootstrapped_stats = [stat_func(data[np.random.choice(len(data), len(data), replace=True)]) 
                          for _ in range(n_bootstraps)]
    
    final_radius = np.mean(bootstrapped_stats)
    return final_radius if not np.isnan(final_radius) and final_radius > 0 else stat_func(data)

def calculate_error_radii_for_subjects(functional_results, analysis_subjects):
    """Calculate measurement error radii for all subjects"""
    radii = {}
    
    for subject_id in analysis_subjects.keys():
        if subject_id not in functional_results:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        radii[subject_id] = {}
        
        print(f"\n{code} - Calculating error radii:")
        
        for category, sessions_data in functional_results[subject_id].items():
            if len(sessions_data) < 2:
                radii[subject_id][category] = 1.0  # Default radius
                print(f"  {category}: 1.0mm (default - insufficient sessions)")
                continue
            
            # Collect peak coordinates for bootstrapping
            pair_peaks = []
            for session, data in sessions_data.items():
                pair_peaks.append({
                    'coord': data['centroid'],
                    'session': session
                })
            
            radius = get_bootstrapped_error_radius(pair_peaks)
            radii[subject_id][category] = radius
            print(f"  {category}: {radius:.2f}mm")
    
    return radii

# Calculate radii for all analysis subjects
print("CALCULATING MEASUREMENT ERROR RADII")
print("="*70)
radii = calculate_error_radii_for_subjects(golarai_functional_final, ANALYSIS_SUBJECTS)
print("✓ Error radii calculated!")

CALCULATING MEASUREMENT ERROR RADII

OTC004 - Calculating error radii:
  face: 7.80mm
  word: 9.35mm
  object: 1.36mm
  house: 5.32mm

nonOTC007 - Calculating error radii:
  face: 3.17mm
  word: 3.89mm
  object: 1.70mm
  house: 13.43mm

OTC008 - Calculating error radii:
  face: 0.94mm
  word: 1.0mm (default - insufficient sessions)
  object: 2.83mm
  house: 4.05mm

OTC010 - Calculating error radii:
  face: 0.41mm
  word: 5.47mm
  object: 0.61mm
  house: 2.73mm

OTC017 - Calculating error radii:
  face: 3.81mm
  word: 18.14mm
  object: 1.89mm
  house: 18.46mm

OTC021 - Calculating error radii:
  face: 0.55mm
  word: 3.96mm
  object: 1.37mm
  house: 5.55mm

nonOTC045 - Calculating error radii:
  face: 2.03mm
  word: 0.36mm
  object: 0.35mm
  house: 1.28mm

nonOTC047 - Calculating error radii:
  face: 0.37mm
  word: 0.56mm
  object: 0.21mm
  house: 2.66mm

nonOTC049 - Calculating error radii:
  face: 0.23mm
  word: 0.49mm
  object: 0.54mm
  house: 0.68mm

nonOTC070 - Calculating error rad

In [10]:
# CELL: DRIFT ANALYSIS (UPDATED)
def calculate_centroid_drift(functional_results, radii, analysis_subjects):
    """Calculate drift between sessions for each category - UPDATED"""
    drift_results = {}
    
    for subject_id, categories in functional_results.items():
        if subject_id not in analysis_subjects:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        group_status = f"{info['group']} {info['patient_status']}"
        
        drift_results[subject_id] = {}
        
        print(f"\n{code} ({group_status}): Centroid Drift Analysis")
        
        for category, sessions_data in categories.items():
            if len(sessions_data) < 2:
                continue
            
            sessions = sorted(sessions_data.keys())
            baseline_session = sessions[0]
            baseline_centroid = sessions_data[baseline_session]['centroid']
            error_radius = radii[subject_id].get(category, 1.0)
            
            drift_results[subject_id][category] = {
                'baseline_session': baseline_session,
                'baseline_centroid': baseline_centroid,
                'error_radius': error_radius,
                'from_baseline_drift': []
            }
            
            # Calculate drift from baseline
            for session in sessions[1:]:
                current_centroid = sessions_data[session]['centroid']
                drift_from_baseline = np.linalg.norm(current_centroid - baseline_centroid)
                
                drift_results[subject_id][category]['from_baseline_drift'].append({
                    'session': session,
                    'distance_mm': drift_from_baseline,
                    'relative_to_error': drift_from_baseline / error_radius
                })
                
                print(f"  {category} ses-{session}: {drift_from_baseline:.2f}mm drift (error radius: {error_radius:.2f}mm)")
    
    return drift_results

# Run drift analysis
print("CALCULATING CENTROID DRIFT")
print("="*70)
drift_data = calculate_centroid_drift(golarai_functional_final, radii, ANALYSIS_SUBJECTS)
print("✓ Drift analysis complete!")

CALCULATING CENTROID DRIFT

OTC004 (OTC patient): Centroid Drift Analysis
  face ses-03: 23.26mm drift (error radius: 7.80mm)
  face ses-05: 24.40mm drift (error radius: 7.80mm)
  face ses-06: 15.05mm drift (error radius: 7.80mm)
  word ses-03: 18.50mm drift (error radius: 9.35mm)
  word ses-05: 1.81mm drift (error radius: 9.35mm)
  word ses-06: 24.41mm drift (error radius: 9.35mm)
  object ses-02: 2.74mm drift (error radius: 1.36mm)
  object ses-03: 4.15mm drift (error radius: 1.36mm)
  object ses-05: 4.18mm drift (error radius: 1.36mm)
  object ses-06: 3.15mm drift (error radius: 1.36mm)
  house ses-02: 2.68mm drift (error radius: 5.32mm)
  house ses-03: 3.18mm drift (error radius: 5.32mm)
  house ses-05: 2.14mm drift (error radius: 5.32mm)
  house ses-06: 15.66mm drift (error radius: 5.32mm)

nonOTC007 (nonOTC patient): Centroid Drift Analysis
  face ses-03: 8.49mm drift (error radius: 3.17mm)
  face ses-04: 3.46mm drift (error radius: 3.17mm)
  word ses-04: 18.53mm drift (error rad

In [11]:
# CELL: BILATERAL VS UNILATERAL CATEGORY ANALYSIS (UPDATED)
def analyze_bilateral_vs_unilateral(drift_data, distinctiveness_results, slope_results, analysis_subjects):
    """
    Comprehensive analysis comparing bilateral vs unilateral categories - UPDATED
    """
    
    # Category groupings
    bilateral_categories = ['object', 'house']
    unilateral_categories = ['face', 'word']
    
    results = {
        'spatial_drift': {},
        'representational_stability': {},
        'reorganization_patterns': {}
    }
    
    print("BILATERAL vs UNILATERAL CATEGORY ANALYSIS")
    print("="*70)
    
    # ============================================================================
    # PART 1: SPATIAL DRIFT COMPARISON
    # ============================================================================
    
    print("\n1. SPATIAL DRIFT ANALYSIS")
    print("-" * 40)
    
    all_bilateral_drift = []
    all_unilateral_drift = []
    
    for subject_id, categories in drift_data.items():
        if subject_id not in analysis_subjects:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        group_status = f"{info['group']} {info['patient_status']}"
        
        print(f"\n{code} ({group_status}):")
        
        subject_bilateral = []
        subject_unilateral = []
        
        for category, data in categories.items():
            if not data['from_baseline_drift']:
                continue
            
            # Get maximum drift from baseline
            max_drift = max([d['distance_mm'] for d in data['from_baseline_drift']])
            mean_drift = np.mean([d['distance_mm'] for d in data['from_baseline_drift']])
            
            if category in bilateral_categories:
                subject_bilateral.append(mean_drift)
                all_bilateral_drift.append(mean_drift)
                print(f"  {category} (bilateral): {mean_drift:.2f}mm avg drift")
            elif category in unilateral_categories:
                subject_unilateral.append(mean_drift)
                all_unilateral_drift.append(mean_drift)
                print(f"  {category} (unilateral): {mean_drift:.2f}mm avg drift")
        
        # Subject-level comparison
        if subject_bilateral and subject_unilateral:
            bilateral_avg = np.mean(subject_bilateral)
            unilateral_avg = np.mean(subject_unilateral)
            print(f"  → Bilateral avg: {bilateral_avg:.2f}mm")
            print(f"  → Unilateral avg: {unilateral_avg:.2f}mm")
            print(f"  → Difference: {unilateral_avg - bilateral_avg:.2f}mm")
    
    # Overall statistics
    if all_bilateral_drift and all_unilateral_drift:
        from scipy.stats import ttest_ind
        
        print(f"\nOVERALL SPATIAL DRIFT COMPARISON:")
        print(f"Bilateral categories: {np.mean(all_bilateral_drift):.2f} ± {np.std(all_bilateral_drift):.2f}mm")
        print(f"Unilateral categories: {np.mean(all_unilateral_drift):.2f} ± {np.std(all_unilateral_drift):.2f}mm")
        
        stat, p_val = ttest_ind(all_bilateral_drift, all_unilateral_drift)
        print(f"t-test: t={stat:.3f}, p={p_val:.3f}")
        
        results['spatial_drift'] = {
            'bilateral_drifts': all_bilateral_drift,
            'unilateral_drifts': all_unilateral_drift,
            'bilateral_mean': np.mean(all_bilateral_drift),
            'unilateral_mean': np.mean(all_unilateral_drift),
            'test_statistic': stat,
            'p_value': p_val
        }
    
    return results

# Note: You'll need to add distinctiveness_results and slope_results analysis first
# For now, run with just drift data:
print("Running bilateral vs unilateral analysis with available data...")

# Simplified version focusing on spatial drift only
bilateral_vs_unilateral_results = analyze_bilateral_vs_unilateral(
    drift_data, {}, {}, ANALYSIS_SUBJECTS  # Empty dicts for missing analyses
)

Running bilateral vs unilateral analysis with available data...
BILATERAL vs UNILATERAL CATEGORY ANALYSIS

1. SPATIAL DRIFT ANALYSIS
----------------------------------------

OTC004 (OTC patient):
  face (unilateral): 20.90mm avg drift
  word (unilateral): 14.91mm avg drift
  object (bilateral): 3.56mm avg drift
  house (bilateral): 5.91mm avg drift
  → Bilateral avg: 4.74mm
  → Unilateral avg: 17.90mm
  → Difference: 13.17mm

nonOTC007 (nonOTC patient):
  face (unilateral): 5.97mm avg drift
  word (unilateral): 18.53mm avg drift
  object (bilateral): 4.67mm avg drift
  house (bilateral): 22.84mm avg drift
  → Bilateral avg: 13.76mm
  → Unilateral avg: 12.25mm
  → Difference: -1.51mm

OTC008 (OTC patient):
  face (unilateral): 4.13mm avg drift
  object (bilateral): 11.37mm avg drift
  house (bilateral): 17.59mm avg drift
  → Bilateral avg: 14.48mm
  → Unilateral avg: 4.13mm
  → Difference: -10.35mm

OTC010 (OTC patient):
  face (unilateral): 3.54mm avg drift
  word (unilateral): 22.07m

In [26]:
# UPDATED: Final analysis that uses the distinctiveness results
def analyze_spatial_representational_coupling_complete(drift_data, distinctiveness_data, analysis_subjects):
    """
    Complete spatial-representational coupling analysis with REAL distinctiveness data
    """
    
    print("FINAL ANALYSIS: SPATIAL-REPRESENTATIONAL COUPLING (COMPLETE)")
    print("="*80)
    
    bilateral_categories = ['object', 'house']
    unilateral_categories = ['face', 'word']
    
    results = {}
    
    for subject_id in analysis_subjects.keys():
        if subject_id not in drift_data:
            continue
            
        info = analysis_subjects[subject_id]
        code = info['code']
        group_status = f"{info['group']} {info['patient_status']}"
        
        print(f"\n{code} ({group_status})")
        print("-" * 50)
        
        results[subject_id] = {
            'spatial_changes': {},
            'representational_changes': {},
            'coupling_data': []
        }
        
        # Calculate change scores for each ROI
        for category in ['face', 'word', 'object', 'house']:
            
            # SPATIAL CHANGE: Mean drift from baseline
            if (category in drift_data.get(subject_id, {}) and 
                drift_data[subject_id][category]['from_baseline_drift']):
                
                spatial_drifts = [d['distance_mm'] for d in drift_data[subject_id][category]['from_baseline_drift']]
                mean_spatial_change = np.mean(spatial_drifts)
            else:
                mean_spatial_change = 0
            
            # REPRESENTATIONAL CHANGE: Change in distinctiveness
            if (subject_id in distinctiveness_data and 
                category in distinctiveness_data[subject_id]):
                
                sessions = sorted(distinctiveness_data[subject_id][category].keys())
                if len(sessions) >= 2:
                    baseline_dist = distinctiveness_data[subject_id][category][sessions[0]]['liu_distinctiveness']
                    final_dist = distinctiveness_data[subject_id][category][sessions[-1]]['liu_distinctiveness']
                    representational_change = abs(final_dist - baseline_dist)
                else:
                    representational_change = 0
            else:
                representational_change = 0
            
            # Store results
            results[subject_id]['spatial_changes'][category] = mean_spatial_change
            results[subject_id]['representational_changes'][category] = representational_change
            
            print(f"  {category.upper()}: spatial={mean_spatial_change:.2f}mm, repr={representational_change:.3f}")
    
    return results

# Re-run the analysis with REAL representational data
final_results_complete = analyze_spatial_representational_coupling_complete(
    drift_data, liu_distinctiveness, ANALYSIS_SUBJECTS
)

# Re-create the table with REAL representational values
results_table_complete = create_spatial_representational_table(final_results_complete, ANALYSIS_SUBJECTS)

print(f"\n✓ NOW we have the complete spatial-representational coupling analysis!")

FINAL ANALYSIS: SPATIAL-REPRESENTATIONAL COUPLING (COMPLETE)

OTC004 (OTC patient)
--------------------------------------------------
  FACE: spatial=20.90mm, repr=0.218
  WORD: spatial=14.91mm, repr=0.082
  OBJECT: spatial=3.56mm, repr=0.099
  HOUSE: spatial=5.91mm, repr=0.487

nonOTC007 (nonOTC patient)
--------------------------------------------------
  FACE: spatial=5.97mm, repr=0.039
  WORD: spatial=18.53mm, repr=0.016
  OBJECT: spatial=4.67mm, repr=0.109
  HOUSE: spatial=22.84mm, repr=0.093

OTC008 (OTC patient)
--------------------------------------------------
  FACE: spatial=4.13mm, repr=0.097
  WORD: spatial=0.00mm, repr=0.000
  OBJECT: spatial=11.37mm, repr=0.748
  HOUSE: spatial=17.59mm, repr=0.035

OTC010 (OTC patient)
--------------------------------------------------
  FACE: spatial=3.54mm, repr=0.111
  WORD: spatial=22.07mm, repr=0.290
  OBJECT: spatial=2.66mm, repr=0.134
  HOUSE: spatial=14.10mm, repr=0.430

OTC017 (OTC patient)
---------------------------------------

In [25]:
# CELL: Simple Clean Analysis (Manual Exclusion)
def analyze_complete_spatial_representational_coupling_simple(results_table):
    """Simple analysis with manual exclusion of insufficient data subjects"""
    
    # Manual exclusion of subjects with insufficient data (all zeros)
    subjects_to_skip = ['OTC079', 'OTC108']  # Add others if needed based on your output
    
    # Filter out summary rows and excluded subjects
    clean_data = results_table[
        (results_table['Category_Type'] != 'Summary') &
        (~results_table['Subject'].isin(subjects_to_skip))
    ].copy()
    
    print("SPATIAL-REPRESENTATIONAL COUPLING ANALYSIS")
    print("="*70)
    print(f"Excluded subjects: {subjects_to_skip}")
    print(f"Analyzing {len(clean_data)} ROI combinations from {clean_data['Subject'].nunique()} subjects")
    
    # Overall correlation
    spatial_vals = clean_data['Spatial_Drift_mm'].values
    repr_vals = clean_data['Representational_Change'].values
    
    overall_corr = np.corrcoef(spatial_vals, repr_vals)[0, 1]
    print(f"\nOVERALL SPATIAL-REPRESENTATIONAL CORRELATION: r = {overall_corr:.3f}")
    
    # Group analysis
    print(f"\nGROUP PATTERNS:")
    print("-" * 40)
    
    for group in ['OTC', 'nonOTC']:
        group_data = clean_data[clean_data['Group'] == group]
        bilateral = group_data[group_data['Category_Type'] == 'Bilateral']
        unilateral = group_data[group_data['Category_Type'] == 'Unilateral']
        
        print(f"\n{group.upper()} PATIENTS:")
        print(f"  Bilateral:  spatial={bilateral['Spatial_Drift_mm'].mean():.1f}mm, repr={bilateral['Representational_Change'].mean():.3f}")
        print(f"  Unilateral: spatial={unilateral['Spatial_Drift_mm'].mean():.1f}mm, repr={unilateral['Representational_Change'].mean():.3f}")
    
    # Statistical tests
    bilateral_all = clean_data[clean_data['Category_Type'] == 'Bilateral']
    unilateral_all = clean_data[clean_data['Category_Type'] == 'Unilateral']
    
    from scipy.stats import ttest_ind
    repr_stat, repr_p = ttest_ind(bilateral_all['Representational_Change'], unilateral_all['Representational_Change'])
    spatial_stat, spatial_p = ttest_ind(bilateral_all['Spatial_Drift_mm'], unilateral_all['Spatial_Drift_mm'])
    
    print(f"\nSTATISTICAL RESULTS:")
    print(f"  Repr change: Bilateral {bilateral_all['Representational_Change'].mean():.3f} vs Unilateral {unilateral_all['Representational_Change'].mean():.3f} (p={repr_p:.3f})")
    print(f"  Spatial drift: Bilateral {bilateral_all['Spatial_Drift_mm'].mean():.1f} vs Unilateral {unilateral_all['Spatial_Drift_mm'].mean():.1f} (p={spatial_p:.3f})")
    print(f"  Overall coupling: r={overall_corr:.3f}")
    
    return clean_data

# Run simple analysis
clean_results = analyze_complete_spatial_representational_coupling_simple(results_table)

SPATIAL-REPRESENTATIONAL COUPLING ANALYSIS
Excluded subjects: ['OTC079', 'OTC108']
Analyzing 56 ROI combinations from 14 subjects

OVERALL SPATIAL-REPRESENTATIONAL CORRELATION: r = nan

GROUP PATTERNS:
----------------------------------------

OTC PATIENTS:
  Bilateral:  spatial=12.4mm, repr=0.000
  Unilateral: spatial=11.5mm, repr=0.000

NONOTC PATIENTS:
  Bilateral:  spatial=4.5mm, repr=0.000
  Unilateral: spatial=7.1mm, repr=0.000

STATISTICAL RESULTS:
  Repr change: Bilateral 0.000 vs Unilateral 0.000 (p=nan)
  Spatial drift: Bilateral 7.4 vs Unilateral 8.7 (p=0.601)
  Overall coupling: r=nan
